### IMPORTS

In [1]:
import subprocess
import sys
import time

In [2]:
import ipynb.fs.full.utils as utils
import ipynb.fs.full.blockchain as blockchain_utils
import ipynb.fs.full.device as device_utils
import ipynb.fs.full.oracle as oracle

### LOGGING BASELINE

In [3]:
def log(msg):
    print('[LAUNCHER] ' + msg)

### LOAD DATA FROM EXTERNAL FILES

In [4]:
settings = utils.load_json('config/settings.json')

In [5]:
latest = utils.load_json('config/latest.json')

In [6]:
device_info = utils.load_yaml('config/identifier.yaml')

In [7]:
log('READ DATA FROM EXTERNAL FILES')

[LAUNCHER] READ DATA FROM EXTERNAL FILES


### CONNECT TO BLOCKCHAIN VIA WEBSOCKET

In [8]:
web3 = blockchain_utils.connect(settings)

CONNECTED WITH GATEWAY!


### EXTRACT THE WHISPER API

In [9]:
shh = web3.geth.shh

### GENERATE A FRESH WHISPER ID

In [10]:
whisper_id = shh.newKeyPair()

In [11]:
log('WHISPER ID GENERATED')

[LAUNCHER] WHISPER ID GENERATED


### CREATE DEVICE OUTLINE

In [12]:
oracle = device_utils.create(device_info)

In [13]:
log('DEVICE ID HASHED')

[LAUNCHER] DEVICE ID HASHED


### SERIALIZE NECESSARY MANAGER CONTRACTS

In [14]:
oracle_manager = blockchain_utils.contract(latest['oraclemanager'], web3, settings)

In [15]:
task_manager = blockchain_utils.contract(latest['taskmanager'], web3, settings)

In [16]:
log('MANAGER CONTRACTS SERIALIZED')

[LAUNCHER] MANAGER CONTRACTS SERIALIZED


### FETCH & SERIALIZE ORACLE CONTRACT

In [17]:
temp_contract = blockchain_utils.contract({
    'address': oracle_manager.read({
        'func': 'fetch_oracle',
        'params': oracle.hash
    }),
    'abi': latest['oracle']['abi']
}, web3, settings)

### VERIFY ORACLE CONTRACT EXISTENCE

In [18]:
if temp_contract.address != '0x0000000000000000000000000000000000000000':
    oracle.set_contract(temp_contract)
    log('ATTACHED ORACLE CONTRACT TO DEVICE')
    
else:
    log('THE ORACLE IS NOT REGISTERED, ABORTING...')
    sys.exit

[LAUNCHER] ATTACHED ORACLE CONTRACT TO DEVICE


### GLOBAL TASK BACKLOG

In [19]:
raw = oracle.read('details')[5]
backlog = utils.filter_backlog(raw)

In [20]:
log('TASKS IN BACKLOG: ' + str(len(backlog)))

[LAUNCHER] TASKS IN BACKLOG: 0


### GLOBAL ACTIVE STATUS 

In [21]:
active = oracle.read('active')

In [22]:
log('CURRENT ACTIVE STATUS: ' + str(active))

[LAUNCHER] CURRENT ACTIVE STATUS: False


### GLOBAL DISCOVERY STATUS

In [23]:
discoverable = oracle.read('discoverable')

In [24]:
log('CURRENT DISCOVERABLE STATUS: ' + str(discoverable))

[LAUNCHER] CURRENT DISCOVERABLE STATUS: False


### GLOBAL DISCOVERY CONFIG

In [25]:
encoded = oracle.read('config')

In [26]:
discovery_config = utils.decode(encoded)

In [27]:
log('DISCOVERY PARAMS FETCHED')

[LAUNCHER] DISCOVERY PARAMS FETCHED


### EVENT LOOPS

In [28]:
def update_details(event):
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global discovery_config
    global backlog
    
    # EXTRACT RELEVANT VALUES
    latest_active = event['args']['active']
    latest_discoverable = event['args']['discoverable']
    latest_config = decode(event['args']['config'])

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        log('ACTIVE STATUS CHANGED TO: ' + str(latest_active))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        log('DISCOVERY CONFIG CHANGED')
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [29]:
def update_middleware():
    
    # PRINT REACTION
    log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [30]:
def perform_task(task):
    
    # SHOW MSG
    log('STARTING TASK: ' + task)

    # PERFORM ORACLE TASK
    result = oracle.perform_task(task)

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, result]
    })

    # SHOW MSG
    log('TASK COMPLETED')

In [31]:
def process_message(event):

    # SERIALIZE EVENT PARAMS
    author = blockchain.toHex(event['sig'])
    payload = blockchain.toText(event['payload'])

    # DECODE THE PAYLOAD
    data = decode(payload)
    
    # REQUIRED KEYS FOR VALID MESSAGE
    required = ['type', 'discovery']

    # DECODED KEYS
    keys = list(data.keys())
    
    # THE REQUEST KEYWORD FOR THE PAYLOAD TYPE
    keyword = 'request'
    
    # IF THE KEYSETS MATCH & THE TYPE IS A REQUEST
    if (required == keys and data['type'] == keyword):
        
        # CHECK MATCHES IN DISCOVERY PARAMS
        discovery_result = compare_discovery(data['discovery'], discovery_config)
        
        # IF EVERYTHING MATCHED
        if (discovery_result.count(False) == 0):
            
            # SHOW MSG
            log('DISCOVERY REQUEST DETECTED')
            
            # ENCODE A JSON RESPONSE
            response = encode({
                'type': 'response',
                'source': payload,
                'device': device.hash
            })
            
            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['topic']['key'],
                'payload': blockchain.toHex(text=response),
                'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### START LISTENING FOR EVENTS

In [32]:
log('AWAITING EVENTS...\n')

[LAUNCHER] AWAITING EVENTS...



### CONTRACT EVENTS

In [33]:
update_event = oracle.event('middleware')

In [34]:
modification_event = oracle.event('modification')

In [35]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

### EVENT LOOP

In [36]:
try:
    while(True):
    
        # FETCH THE GLOBAL PARAMS
        global backlog
        global active
        global discoverable

        # VARIABLE MODIFICATION EVENT
        for event in modification_event.get_new_entries():
            update_details(event)

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)

# WHEN THE PROCESS IS KILLED...
except KeyboardInterrupt:
    print('\nThe process was manually stopped...')
    pass


The process was manually stopped...
